In [68]:
import pandas as pd
import glob
import re

> Load all the data into one dataframe

In [64]:
# Using glob  & regex to get all the files appropriate files in the directory
egrid_files = glob.glob("../data/rmr_62/egrid_data/e[Gg][Rr][Ii][Dd]*.xls*")

In [72]:
# Reading the egrid files into a pandas dataframe, only keep the tab we need
egrid_df = pd.DataFrame()
for file in egrid_files:
    match = re.search(r"\d{4}", file)
    year = match.group()

    df = pd.read_excel(file, sheet_name=f"PLNT{year[-2]}{year[-1]}", skiprows=1)
    df["Year"] = year

    egrid_df = pd.concat([egrid_df, df])

del df  # Clean up
egrid_df.loc[:, 'AverageCap'] = egrid_df['NAMECAP'] * egrid_df['CAPFAC']

## Column name descriptions

| Column    | Description |
|-----------|-------------|
| PNAME     | Plan Name |
| PLPRMFL   | Primary Fuel |
| CAPFAC    | Capacity Facoty |
| NAMEPCAP  | Nameplate Capacity |
| PLNGENAN  | Plant annual net genration (MWh) |
| PLNOXAN   | Plant annual Nox Emission (tonnes) |
| PLSO2AN   | Plant annual So2 Emission (tonnes) |
| PLCO2AN   | Plant annual Co2 Emission (tonnes) |
| Year      | Data for Year |
| TotEm     | Total aanual plant emissions (tonnes) |
| AverageCap | Name plate capacity X Capacity Factor |

In [86]:
egrid_df = egrid_df[
    [
        "PNAME",
        "PLPRMFL",
        "AverageCap",
        "PLNGENAN",
        "PLNOXAN",
        "PLSO2AN",
        "PLCO2AN",
        "Year",
    ]
]

c = "TotEm"
c2 = "PLNOXAN"
egrid_df.loc[:, c] = egrid_df[c2] + egrid_df["PLSO2AN"] + egrid_df["PLCO2AN"]

In [90]:
renewable_fuels = ["WAT", "WND", "SUN", "GEO", "HYD", "BIO", "LFG", "OBG", "BG"]
egrid_df_renewable = egrid_df[egrid_df["PLPRMFL"].isin(renewable_fuels)]

cols = ["AverageCap", "PLNGENAN", "PLNOXAN", "PLSO2AN", "PLCO2AN", "TotEm"]
egrid_df_renewable_by_year = egrid_df_renewable.groupby("Year")[cols].sum()

In [91]:
egrid_df_renewable_by_year

,AverageCap,PLNGENAN,PLNOXAN,PLSO2AN,PLCO2AN,TotEm
Year,,,,,,
2004,33096.641060,2.899262e+08,3950.279900,3.875700,8.328858e+04,8.724273e+04
2005,34249.130700,3.000201e+08,3846.696200,1084.371900,1.225320e+06,1.230251e+06
2007,33658.553750,2.948496e+08,11478.881800,1012.827000,6.201832e+05,6.326749e+05
2009,41573.026200,3.641773e+08,441.643200,1241.760700,5.524852e+05,5.541686e+05
2010,42472.819900,3.720530e+08,497.985100,1166.787700,5.442184e+05,5.458832e+05
2012,49991.832555,4.379285e+08,545.358774,1131.957624,6.517902e+05,6.534676e+05
2014,55333.652700,4.785271e+08,84307.102000,4958.340000,1.410129e+06,1.499344e+06
2016,63678.903164,5.508966e+08,77873.068000,4724.678000,1.366063e+06,1.442632e+06
2018,74864.143193,6.495603e+08,14006.368000,4544.503000,1.371638e+06,1.390189e+06


In [60]:
emissions_df = pd.read_csv("../data/rmr_62/emissions_data/emissions_epa.csv")

In [61]:
emissions_df

,Year,Carbon dioxide,Methane,Nitrous oxide,"HFCs, PFCs, SF6, and NF3"
0,1990,"5,120.96",868.661,406.281,91.432
1,1991,"5,062.96",874.355,397.595,83.500
2,1992,"5,174.23",874.450,398.820,87.383
3,1993,"5,272.85",863.427,415.889,87.358
4,1994,"5,364.28",871.454,418.494,90.940
5,1995,"5,425.26",865.942,422.923,107.809
6,1996,"5,612.98",859.731,432.744,118.072
7,1997,"5,688.14",845.140,421.073,124.851
8,1998,"5,733.28",830.142,422.535,138.727
9,1999,"5,803.92",812.532,416.449,136.987
